# Parte 4: Tornando o Modelo Útil - Deploy

## Projeto: Previsão de Churn de Clientes de Telecomunicações

**Objetivo:** Demonstrar que o modelo não é apenas um exercício acadêmico, mas pode ser reutilizado para fazer novas previsões.

---

In [16]:
import pandas as pd
import numpy as np
import joblib
import warnings
warnings.filterwarnings('ignore')

# Modelos (para recriar o melhor modelo)
from sklearn.ensemble import GradientBoostingClassifier

## 4.1. Salvando o Modelo Treinado

Primeiro, vamos recarregar os dados de treino e treinar novamente nosso melhor modelo, o **Gradient Boosting**, para garantir que temos a versão final pronta para ser salva.

In [17]:
# Carregar dados de treino completos
X_train = pd.read_csv('../data/processed/X_train.csv')
y_train = pd.read_csv('../data/processed/y_train.csv').values.ravel()

print("Dados de treino carregados:")
print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")

Dados de treino carregados:
X_train: (5625, 30)
y_train: (5625,)


In [18]:
# Instanciar e treinar o modelo final
final_model = GradientBoostingClassifier(n_estimators=100, random_state=42, max_depth=5)

print("Treinando o modelo final (Gradient Boosting)...")
final_model.fit(X_train, y_train)
print("Modelo treinado com sucesso!")

Treinando o modelo final (Gradient Boosting)...
Modelo treinado com sucesso!


In [19]:
# Salvar o modelo treinado usando joblib
model_filename = '../modelo_final.pkl'

print(f"Salvando modelo em: {model_filename}")
joblib.dump(final_model, model_filename)
print(f"Modelo salvo com sucesso!")

Salvando modelo em: ../modelo_final.pkl
Modelo salvo com sucesso!


## 4.2. Carregando e Utilizando o Modelo

Agora, vamos demonstrar como carregar o modelo salvo e usá-lo para fazer uma previsão em um **exemplo de um novo dado** (um cliente que o modelo nunca viu).

In [20]:
# Carregar o modelo a partir do arquivo
print(f"Carregando modelo de: {model_filename}")
loaded_model = joblib.load(model_filename)
print("Modelo carregado com sucesso!")

# Verificar se o modelo carregado é o mesmo
print(f"\nModelo carregado: {loaded_model}")

Carregando modelo de: ../modelo_final.pkl
Modelo carregado com sucesso!

Modelo carregado: GradientBoostingClassifier(max_depth=5, random_state=42)


### Criando um Exemplo de Novo Dado

Vamos criar o perfil de um **novo cliente fictício**. Este cliente tem características que, com base na nossa análise exploratória, podem indicar um **alto risco de churn**:

- **Contrato:** Mensal (Month-to-month)
- **Tempo de permanência (tenure):** Baixo (ex: 3 meses)
- **Serviço de internet:** Fibra óptica (geralmente mais cara)
- **Pagamento:** Cheque eletrônico (associado a maior churn)
- **Sem serviços de proteção:** Sem segurança online, sem backup, sem suporte técnico

In [21]:
# Criar um dicionário com os dados do novo cliente
new_customer_data = {
    'gender': 'Female',
    'SeniorCitizen': 0,
    'Partner': 'No',
    'Dependents': 'No',
    'tenure': 3,
    'PhoneService': 'Yes',
    'MultipleLines': 'No',
    'InternetService': 'Fiber optic',
    'OnlineSecurity': 'No',
    'OnlineBackup': 'No',
    'DeviceProtection': 'No',
    'TechSupport': 'No',
    'StreamingTV': 'Yes',
    'StreamingMovies': 'Yes',
    'Contract': 'Month-to-month',
    'PaperlessBilling': 'Yes',
    'PaymentMethod': 'Electronic check',
    'MonthlyCharges': 90.0,
    'TotalCharges': 270.0
}

# Converter para DataFrame
new_customer_df = pd.DataFrame([new_customer_data])

print("="*70)
print("DADOS DO NOVO CLIENTE (FORMATO ORIGINAL)")
print("="*70)
new_customer_df.T

DADOS DO NOVO CLIENTE (FORMATO ORIGINAL)


,0
gender,Female
SeniorCitizen,0
Partner,No
Dependents,No
tenure,3
PhoneService,Yes
MultipleLines,No
InternetService,Fiber optic
OnlineSecurity,No
OnlineBackup,No


### Pré-processamento do Novo Dado

O modelo foi treinado com dados pré-processados (One-Hot Encoding). Portanto, precisamos aplicar **exatamente a mesma transformação** ao novo dado antes de fazer a previsão.

In [22]:
# Carregar as colunas do modelo treinado para garantir consistência
model_columns = X_train.columns

# Aplicar One-Hot Encoding
new_customer_encoded = pd.get_dummies(new_customer_df)

# Reindexar para garantir que as colunas sejam exatamente as mesmas do treino
# Colunas faltantes no novo dado serão preenchidas com 0
new_customer_processed = new_customer_encoded.reindex(columns=model_columns, fill_value=0)

print("="*70)
print("DADOS DO NOVO CLIENTE (APOS PRE-PROCESSAMENTO)")
print("="*70)
print(f"Dimensoes: {new_customer_processed.shape}")

# Visualizar algumas colunas para verificar
print("\nExemplo de colunas processadas:")
cols_to_show = ['tenure', 'MonthlyCharges', 'TotalCharges', 'Contract_One year', 'InternetService_Fiber optic', 'PaymentMethod_Credit card (automatic)']
display(new_customer_processed[cols_to_show].T)

DADOS DO NOVO CLIENTE (APOS PRE-PROCESSAMENTO)
Dimensoes: (1, 30)

Exemplo de colunas processadas:


,0
tenure,3
MonthlyCharges,90.0
TotalCharges,270.0
Contract_One year,0
InternetService_Fiber optic,True
PaymentMethod_Credit card (automatic),0


### Fazendo a Previsão

Agora que o dado está no formato correto, podemos usar o método `predict()` para obter a classificação e `predict_proba()` para obter a probabilidade de churn.

In [23]:
# Fazer a previsão
prediction = loaded_model.predict(new_customer_processed)
prediction_proba = loaded_model.predict_proba(new_customer_processed)

# Interpretar o resultado
churn_status = "Sim" if prediction[0] == 1 else "Nao"
churn_probability = prediction_proba[0][1] * 100

print("="*70)
print("RESULTADO DA PREVISAO PARA O NOVO CLIENTE")
print("="*70)

print(f"Previsao de Churn: {churn_status}")
print(f"Probabilidade de Churn: {churn_probability:.2f}%")

# Explicação do resultado
print("\n" + "-"*70)
print("EXPLICACAO:")
print("-"*70)
if churn_status == "Sim":
    print(f"O modelo previu que este cliente tem uma alta probabilidade ({churn_probability:.2f}%) de cancelar o servico.")
    print("Isso se deve a uma combinacao de fatores de risco, como contrato mensal, baixo tempo de permanencia e uso de servicos que indicam maior sensibilidade a preco.")
    print("\nACAO RECOMENDADA: A equipe de retencao deve entrar em contato com este cliente proativamente para oferecer um beneficio, como um desconto na fatura ou um upgrade de servico, para incentiva-lo a permanecer.")
else:
    print(f"O modelo previu que este cliente tem uma baixa probabilidade ({churn_probability:.2f}%) de cancelar o servico.")
    print("As caracteristicas do cliente indicam que ele esta satisfeito com o servico e nao apresenta risco iminente de churn.")
    print("\nACAO RECOMENDADA: Nenhuma acao de retencao e necessaria no momento. Monitorar o cliente em futuras analises.")

RESULTADO DA PREVISAO PARA O NOVO CLIENTE
Previsao de Churn: Sim
Probabilidade de Churn: 79.44%

----------------------------------------------------------------------
EXPLICACAO:
----------------------------------------------------------------------
O modelo previu que este cliente tem uma alta probabilidade (79.44%) de cancelar o servico.
Isso se deve a uma combinacao de fatores de risco, como contrato mensal, baixo tempo de permanencia e uso de servicos que indicam maior sensibilidade a preco.

ACAO RECOMENDADA: A equipe de retencao deve entrar em contato com este cliente proativamente para oferecer um beneficio, como um desconto na fatura ou um upgrade de servico, para incentiva-lo a permanecer.


## Conclusão da Parte 4

Demonstramos com sucesso o ciclo completo:

1. **Salvamos** o modelo treinado em um arquivo (`modelo_final.pkl`).
2. **Carregamos** o modelo a partir do arquivo.
3. **Criamos** um novo dado de cliente, simulando um cenário real.
4. **Aplicamos** o mesmo pré-processamento dos dados de treino.
5. **Utilizamos** o modelo para fazer uma previsão acionável, incluindo a probabilidade.

Isso prova que o modelo é uma ferramenta funcional que pode ser integrada a sistemas de CRM ou outras aplicações de negócio para automatizar a identificação de clientes em risco de churn.